In [1]:
import sys
sys.path

['/opt/homebrew/anaconda3/envs/wy_test/lib/python310.zip',
 '/opt/homebrew/anaconda3/envs/wy_test/lib/python3.10',
 '/opt/homebrew/anaconda3/envs/wy_test/lib/python3.10/lib-dynload',
 '',
 '/Users/kummy/.local/lib/python3.10/site-packages',
 '/opt/homebrew/anaconda3/envs/wy_test/lib/python3.10/site-packages']

In [2]:
"""
    pygments.regexopt
    ~~~~~~~~~~~~~~~~~

    An algorithm that generates optimized regexes for matching long lists of
    literal strings.

    :copyright: Copyright 2006-2023 by the Pygments team, see AUTHORS.
    :license: BSD, see LICENSE for details.
"""

import re
from re import escape
from os.path import commonprefix
from itertools import groupby
from operator import itemgetter

CS_ESCAPE = re.compile(r'[\[\^\\\-\]]')
FIRST_ELEMENT = itemgetter(0)


def make_charset(letters):
    return '[' + CS_ESCAPE.sub(lambda m: '\\' + m.group(), ''.join(letters)) + ']'


def regex_opt_inner(strings, open_paren):
    """Return a regex that matches any string in the sorted list of strings."""
    close_paren = open_paren and ')' or ''
    # print strings, repr(open_paren)
    if not strings:
        # print '-> nothing left'
        return ''
    first = strings[0]
    if len(strings) == 1:
        # print '-> only 1 string'
        return open_paren + escape(first) + close_paren
    if not first:
        # print '-> first string empty'
        return open_paren + regex_opt_inner(strings[1:], '(?:') \
            + '?' + close_paren
    if len(first) == 1:
        # multiple one-char strings? make a charset
        oneletter = []
        rest = []
        for s in strings:
            if len(s) == 1:
                oneletter.append(s)
            else:
                rest.append(s)
        if len(oneletter) > 1:  # do we have more than one oneletter string?
            if rest:
                # print '-> 1-character + rest'
                return open_paren + regex_opt_inner(rest, '') + '|' \
                    + make_charset(oneletter) + close_paren
            # print '-> only 1-character'
            return open_paren + make_charset(oneletter) + close_paren
    prefix = commonprefix(strings)
    if prefix:
        plen = len(prefix)
        # we have a prefix for all strings
        # print '-> prefix:', prefix
        return open_paren + escape(prefix) \
            + regex_opt_inner([s[plen:] for s in strings], '(?:') \
            + close_paren
    # is there a suffix?
    strings_rev = [s[::-1] for s in strings]
    suffix = commonprefix(strings_rev)
    if suffix:
        slen = len(suffix)
        # print '-> suffix:', suffix[::-1]
        return open_paren \
            + regex_opt_inner(sorted(s[:-slen] for s in strings), '(?:') \
            + escape(suffix[::-1]) + close_paren
    # recurse on common 1-string prefixes
    # print '-> last resort'
    return open_paren + \
        '|'.join(regex_opt_inner(list(group[1]), '')
                 for group in groupby(strings, lambda s: s[0] == first[0])) \
        + close_paren


def regex_opt(strings, prefix='', suffix=''):
    """Return a compiled regex that matches any string in the given list.

    The strings to match must be literal strings, not regexes.  They will be
    regex-escaped.

    *prefix* and *suffix* are pre- and appended to the final regex.
    """
    strings = sorted(strings)
    return prefix + regex_opt_inner(strings, '(') + suffix


In [3]:
import re
text = '123abc456'
re.match(r'abc', text)
re.search(r'abc',text)
re.findall(r'abc', text)


['abc']

In [4]:
r = re.search(r'abc', text)
r.group()

'abc'

In [5]:
text1 = 'abcd'
re.match(r'\w\w', text1)

re.search(r'\w\w', text1).group()

'ab'

In [6]:
re.findall(r'\w\w', text1)
# re.findall(r'\w\w', text1).group()

['ab', 'cd']

In [7]:
# 대괄화 [AB] == A or B

re.findall(r'appl[ea]', 'apple appla appli')

['apple', 'appla']

In [8]:
re.findall(r'appl[^ea]', 'apple appla appli applo')

['appli', 'applo']

In [9]:
text = '2002년 12월 19일 기사입니다.'
result1 = re.search(r'(\d{4})년 (\d{1,2})월 (\d{1,2})일', text)
result1.groups()

('2002', '12', '19')

In [10]:
result1.group(1)

'2002'

In [11]:
text = '2002년 12월 19일 기사입니다.'
result1 = re.search(r'(\d{2})년 (\d{1,2})월 (\d{1,2})일', text)
result1.groups()

('02', '12', '19')

In [12]:
re.findall(r'apple|banana', 'apple banana carrot')

['apple', 'banana']

In [ ]:
re.search(r'Licence: yes|no', 'License: no')
## 뒷 부분에 입력된 No 부분의 자릿수만 찾아서 리턴, 9~11번쨰



<re.Match object; span=(9, 11), match='no'>